# Removing Artifacts
ICA-AROMA (Independent Component Analysis - Automatic Removal of Motion Artifacts) is a data-driven method used in functional MRI (fMRI) to identify and remove motion-related noise from brain imaging data. By applying Independent Component Analysis, it decomposes fMRI data into independent components and uses a robust set of four temporal and spatial features to classify and eliminate motion artifacts without requiring classifier retraining. This preserves the data’s temporal structure and enhances the accuracy of functional connectivity analyses, making it effective for both resting-state and task-based fMRI studies.

In addition, to preprocess the resting-state functional MRI (fMRI) data and mitigate the influence of non-neuronal signals, we utilized the fsl_regfilt tool from the FSL (FMRIB Software Library) to regress out signals associated with white matter (WM), cerebrospinal fluid (CSF), and the global brain signal. This step involved extracting time series from WM and CSF regions using predefined masks, followed by regression to remove their contributions from the data. Additionally, the global signal was regressed to minimize the impact of physiological noise, such as respiratory and cardiac effects, thereby enhancing the specificity of functional connectivity analyses.

#### Link for ICA-AROMA: https://github.com/maartenmennes/ICA-AROMA

-----------------------------------------------------------
Script written by Sepideh Tabrik & Mehdi Behroozi
<br>
Biopsychology, 
<br>
Ruhr-University Bochum, Bochum, Germany
<br>
(2022.03)

-----------------------------------------------------------

In [ ]:
import glob
import os
import pandas as pd
import numpy as np

data_path = '/mnt/d/Data/Human/ObjectCategorization/analysis2'
bold_dirs = glob.glob(f'{data_path}/sub-*/ses-*/func/rest*/*intnorm.nii.gz')

ICA_AROMA_dir = '/home/mehdi/ICA-AROMA'

def run_command(command):
    """Execute a shell command with error handling."""
    exit_code = os.system(command)
    if exit_code != 0:
        print(f"Error executing: {command}")

for curr_bold in bold_dirs:
    sub_dir = os.path.dirname(curr_bold)
    os.chdir(sub_dir)
    print(f'Current file is: {sub_dir}')

    # Run ICA-AROMA
    print('ICA-AROMA is running...')
    run_command(f'python {ICA_AROMA_dir}/ICA_AROMA.py -in {curr_bold} -out {sub_dir}/ICA_AROMA '
                f'-mc {sub_dir}/mc/bold_mcf.par '
                f'-affmat {sub_dir}/reg/example_func2highres.mat '
                f'-warp {sub_dir}/reg/highres2standard_warp.nii.gz')

    # White matter and CSF regression
    for tissue in ["CSF", "WM", "reg/example_func_brain_mask"]:
        run_command(f'fslmeants -i ICA_AROMA/denoised_func_data_nonaggr -o {tissue.split("/")[-1].lower()}.txt -m {sub_dir}/{tissue}.nii.gz')

    # Load data
    csf = pd.read_csv("csf.txt", sep=" ", header=None)[0]
    wm = pd.read_csv("wm.txt", sep=" ", header=None)[0]
    gs = pd.read_csv("example_func_brain_mask.txt", sep=" ", header=None)[0]

    # Stack and save artifacts
    artifacts = np.column_stack((csf, wm, gs))
    np.savetxt("artifacts_csf_wm_gs.txt", artifacts, fmt="%f")

    # Apply regression filtering
    run_command('fsl_regfilt -i ICA_AROMA/denoised_func_data_nonaggr -d artifacts_csf_wm_gs.txt -f "1,2,3" -o bold_mcf_st_smoothed_intnorm_denoised')
